In [1]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
gender = pd.read_csv('공공데이터/한국토지주택공사_임대주택 단지별 연령대별 성별정보_20210511.csv', encoding='cp949')
given_gender = pd.read_csv('data/age_gender_info.csv')
display(train)
display(test)
display(gender)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


,단지_일련번호,임대주택유형,공급기관명,주택명,주택유형,도로명주소,총세대수,10대미만(여자),10대미만(남자),10대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,30485277,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 19,9.0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,30485278,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 7-2,8.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,30485279,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 7-4,7.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,30485280,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로52번길 9-11,10.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30485281,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로74번길 19,8.0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19450,30855821,매입임대,NaN,매입다가구(전북김제시),다가구주택,NaN,0.0,1,4,2,...,0,0,1,0,0,0,0,0,0,0
19451,30855838,매입임대,NaN,매입다가구(전북남원시),다가구주택,NaN,0.0,0,1,2,...,0,0,0,1,0,1,1,0,0,0
19452,30855839,매입임대,NaN,매입다가구(전북남원시),다가구주택,NaN,0.0,1,3,2,...,2,2,0,0,1,1,0,0,0,0
19453,31284963,건설임대,NaN,양주옥정 A-20(2)(공임리츠),아파트,NaN,619.0,99,114,134,...,30,36,9,9,12,0,0,0,0,0


In [3]:
train[['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']] = np.nan

test[['10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']] = np.nan

train1 = train
test1 = test

In [4]:
given_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   지역         16 non-null     object 
 1   10대미만(여자)  16 non-null     float64
 2   10대미만(남자)  16 non-null     float64
 3   10대(여자)    16 non-null     float64
 4   10대(남자)    16 non-null     float64
 5   20대(여자)    16 non-null     float64
 6   20대(남자)    16 non-null     float64
 7   30대(여자)    16 non-null     float64
 8   30대(남자)    16 non-null     float64
 9   40대(여자)    16 non-null     float64
 10  40대(남자)    16 non-null     float64
 11  50대(여자)    16 non-null     float64
 12  50대(남자)    16 non-null     float64
 13  60대(여자)    16 non-null     float64
 14  60대(남자)    16 non-null     float64
 15  70대(여자)    16 non-null     float64
 16  70대(남자)    16 non-null     float64
 17  80대(여자)    16 non-null     float64
 18  80대(남자)    16 non-null     float64
 19  90대(여자)    16 non-null     float64
 20  90대(남자)    1

In [5]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [6]:
train1.drop_duplicates(['단지코드'], inplace=True)
test1.drop_duplicates(['단지코드'], inplace=True)
train1.reset_index(drop=True, inplace=True)
test1.reset_index(drop=True, inplace=True)
display(train1)
display(test1)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C1407,1216,아파트,대전광역시,국민임대,30.95,288,13.0,A,15620000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C1945,755,아파트,경기도,국민임대,39.72,120,6.0,B,15607000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C1470,696,아파트,전라북도,국민임대,36.55,228,14.0,A,12015000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,C2586,90,아파트,제주특별자치도,행복주택,16.64,12,7.0,J,16800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,C2035,492,아파트,강원도,국민임대,29.53,132,24.0,A,10000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,C2020,40,아파트,부산광역시,행복주택,16.80,20,7.0,L,18000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,C2437,90,아파트,충청북도,영구임대,24.83,90,12.0,C,10346000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,H,22830000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C1456,619,아파트,부산광역시,국민임대,33.40,82,18.0,A,19706000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C1840,593,아파트,전라북도,국민임대,39.57,253,7.0,A,14418000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C1332,1297,아파트,경기도,국민임대,39.99,282,11.0,H,28598000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,C2456,349,아파트,제주특별자치도,국민임대,26.44,24,17.0,H,6992000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,C1266,596,아파트,충청북도,국민임대,26.94,164,35.0,H,8084000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,C1267,675,아파트,경상남도,국민임대,24.87,28,38.0,H,6882000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# 세대구성원 결측치 채우는 for문, 지역에 맞춰 총세대수 * given_gender
for i in train1.index:
    if train1.iloc[i].isnull()["10대미만(여자)"] == True:
        a = given_gender[given_gender['지역'] == train1.iloc[i,3]]
        for num in range(len(a.columns[1:])):
            train1.iloc[i, num+15] = round(train1.iloc[i,1] * a.iloc[0, num+1])
    else:
        pass
train1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,...,74.0,57.0,43.0,27.0,26.0,10.0,5.0,1.0,0.0,0.0
1,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,...,48.0,38.0,26.0,18.0,15.0,6.0,3.0,1.0,0.0,0.0
2,C1407,1216,아파트,대전광역시,국민임대,30.95,288,13.0,A,15620000,...,108.0,85.0,62.0,45.0,39.0,17.0,8.0,2.0,0.0,0.0
3,C1945,755,아파트,경기도,국민임대,39.72,120,6.0,B,15607000,...,56.0,44.0,32.0,25.0,19.0,9.0,4.0,1.0,0.0,0.0
4,C1470,696,아파트,전라북도,국민임대,36.55,228,14.0,A,12015000,...,53.0,47.0,36.0,25.0,24.0,9.0,5.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,C2586,90,아파트,제주특별자치도,행복주택,16.64,12,7.0,J,16800000,...,7.0,5.0,4.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0
419,C2035,492,아파트,강원도,국민임대,29.53,132,24.0,A,10000000,...,44.0,34.0,23.0,16.0,16.0,6.0,4.0,1.0,0.0,0.0
420,C2020,40,아파트,부산광역시,행복주택,16.80,20,7.0,L,18000000,...,4.0,3.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0
421,C2437,90,아파트,충청북도,영구임대,24.83,90,12.0,C,10346000,...,7.0,6.0,4.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0


In [8]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          150 non-null    object 
 1   총세대수                          150 non-null    int64  
 2   임대건물구분                        150 non-null    object 
 3   지역                            150 non-null    object 
 4   공급유형                          150 non-null    object 
 5   전용면적                          150 non-null    float64
 6   전용면적별세대수                      150 non-null    int64  
 7   공가수                           150 non-null    float64
 8   자격유형                          149 non-null    object 
 9   임대보증금                         150 non-null    object 
 10  임대료                           150 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  145 non-null    float64
 12  도보 10분거리 내 버스정류장 수            150 non-null    float64
 13  단지내주차

In [9]:
# 세대구성원 결측치 채우는 for문, 지역에 맞춰 총세대수 * given_gender
for i in test1.index:
    if test1.iloc[i].isnull()["10대미만(여자)"] == True:
        a = given_gender[given_gender['지역'] == test1.iloc[i,3]]
        for num in range(len(a.columns[1:])):
            test1.iloc[i, num+14] = round(test1.iloc[i,1] * a.iloc[0, num+1])
    else:
        pass
test1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,...,56.0,44.0,32.0,25.0,19.0,9.0,4.0,1.0,0.0,0.0
1,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,H,22830000,...,101.0,79.0,57.0,44.0,34.0,17.0,7.0,2.0,0.0,0.0
2,C1456,619,아파트,부산광역시,국민임대,33.40,82,18.0,A,19706000,...,68.0,53.0,49.0,33.0,30.0,13.0,5.0,1.0,0.0,0.0
3,C1840,593,아파트,전라북도,국민임대,39.57,253,7.0,A,14418000,...,45.0,40.0,30.0,21.0,21.0,7.0,4.0,1.0,0.0,0.0
4,C1332,1297,아파트,경기도,국민임대,39.99,282,11.0,H,28598000,...,96.0,76.0,55.0,42.0,33.0,16.0,7.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,C2456,349,아파트,제주특별자치도,국민임대,26.44,24,17.0,H,6992000,...,26.0,19.0,17.0,12.0,9.0,4.0,2.0,0.0,0.0,0.0
146,C1266,596,아파트,충청북도,국민임대,26.94,164,35.0,H,8084000,...,45.0,37.0,25.0,16.0,17.0,7.0,3.0,1.0,0.0,0.0
147,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,-,...,11.0,8.0,6.0,4.0,4.0,2.0,1.0,0.0,0.0,0.0
148,C1267,675,아파트,경상남도,국민임대,24.87,28,38.0,H,6882000,...,59.0,47.0,33.0,22.0,18.0,8.0,3.0,1.0,0.0,0.0


In [10]:
train1.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수', '10대미만(여자)', '10대미만(남자)',
       '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)', '30대(여자)', '30대(남자)',
       '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)',
       '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)', '90대(여자)', '90대(남자)',
       '100대(여자)', '100대(남자)'],
      dtype='object')

In [11]:
test1.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '10대미만(여자)', '10대미만(남자)', '10대(여자)',
       '10대(남자)', '20대(여자)', '20대(남자)', '30대(여자)', '30대(남자)', '40대(여자)',
       '40대(남자)', '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)', '70대(여자)',
       '70대(남자)', '80대(여자)', '80대(남자)', '90대(여자)', '90대(남자)', '100대(여자)',
       '100대(남자)'],
      dtype='object')

In [12]:
train1 = train1[['단지코드','10대미만(여자)', '10대미만(남자)',
       '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)', '30대(여자)', '30대(남자)',
       '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)',
       '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)', '90대(여자)', '90대(남자)',
       '100대(여자)', '100대(남자)']]
test1 = test1[['단지코드','10대미만(여자)', '10대미만(남자)',
       '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)', '30대(여자)', '30대(남자)',
       '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)',
       '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)', '90대(여자)', '90대(남자)',
       '100대(여자)', '100대(남자)']]

In [13]:
train1

,단지코드,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,27.0,30.0,51.0,55.0,54.0,61.0,48.0,45.0,75.0,...,74.0,57.0,43.0,27.0,26.0,10.0,5.0,1.0,0.0,0.0
1,C2515,15.0,15.0,29.0,30.0,35.0,38.0,31.0,31.0,42.0,...,48.0,38.0,26.0,18.0,15.0,6.0,3.0,1.0,0.0,0.0
2,C1407,34.0,35.0,49.0,52.0,74.0,78.0,83.0,81.0,91.0,...,108.0,85.0,62.0,45.0,39.0,17.0,8.0,2.0,0.0,0.0
3,C1945,29.0,30.0,40.0,41.0,44.0,45.0,55.0,52.0,63.0,...,56.0,44.0,32.0,25.0,19.0,9.0,4.0,1.0,0.0,0.0
4,C1470,20.0,20.0,42.0,42.0,46.0,49.0,36.0,32.0,54.0,...,53.0,47.0,36.0,25.0,24.0,9.0,5.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,C2586,3.0,3.0,5.0,6.0,6.0,6.0,5.0,4.0,7.0,...,7.0,5.0,4.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0
419,C2035,14.0,15.0,25.0,26.0,30.0,31.0,29.0,27.0,37.0,...,44.0,34.0,23.0,16.0,16.0,6.0,4.0,1.0,0.0,0.0
420,C2020,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,...,4.0,3.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0
421,C2437,3.0,3.0,5.0,6.0,5.0,6.0,5.0,5.0,7.0,...,7.0,6.0,4.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0


In [14]:
test1

,단지코드,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,29.0,30.0,40.0,41.0,44.0,45.0,55.0,52.0,63.0,...,56.0,44.0,32.0,25.0,19.0,9.0,4.0,1.0,0.0,0.0
1,C1128,51.0,53.0,71.0,73.0,79.0,81.0,98.0,93.0,113.0,...,101.0,79.0,57.0,44.0,34.0,17.0,7.0,2.0,0.0,0.0
2,C1456,14.0,14.0,20.0,22.0,33.0,35.0,29.0,30.0,38.0,...,68.0,53.0,49.0,33.0,30.0,13.0,5.0,1.0,0.0,0.0
3,C1840,17.0,17.0,35.0,36.0,39.0,42.0,31.0,28.0,46.0,...,45.0,40.0,30.0,21.0,21.0,7.0,4.0,1.0,0.0,0.0
4,C1332,49.0,51.0,68.0,70.0,76.0,78.0,94.0,89.0,108.0,...,96.0,76.0,55.0,42.0,33.0,16.0,7.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,C2456,12.0,13.0,21.0,22.0,24.0,24.0,18.0,17.0,28.0,...,26.0,19.0,17.0,12.0,9.0,4.0,2.0,0.0,0.0,0.0
146,C1266,21.0,21.0,36.0,37.0,36.0,39.0,36.0,34.0,49.0,...,45.0,37.0,25.0,16.0,17.0,7.0,3.0,1.0,0.0,0.0
147,C2152,3.0,4.0,6.0,6.0,7.0,8.0,7.0,7.0,9.0,...,11.0,8.0,6.0,4.0,4.0,2.0,1.0,0.0,0.0,0.0
148,C1267,18.0,18.0,36.0,38.0,44.0,48.0,38.0,39.0,52.0,...,59.0,47.0,33.0,22.0,18.0,8.0,3.0,1.0,0.0,0.0


In [16]:
train1.to_csv('data/정제된 데이터/트레인_단지_세대원.csv', encoding='cp949')
test1.to_csv('data/정제된 데이터/테스트_단지_세대원.csv', encoding='cp949')